In [15]:
import json
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

In [2]:
# Step 1: Load JSON Data
json_file = 'AMAZON_FASHION_5.json'
df = pd.read_json(json_file, lines=True)

# Step 2: Convert to CSV
csv_file = 'AMAZON_FASHION_5.csv'
df.to_csv(csv_file, index=False)

# Optional: Check the first few rows of the CSV file
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
1,5,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Black (3746...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
2,5,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
3,5,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue (37867...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
4,5,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3171,5,True,"07 2, 2018",A2077NII5H62R2,B005AGO4LU,"{'Size:': ' 8.5 B(M) US', 'Color:': ' Green Gl...",Amazon Customer,Perfect fit!,Five Stars,1530489600,NaN,NaN
3172,5,True,"06 28, 2018",A2IBS6PIPAGAB5,B005AGO4LU,"{'Size:': ' 5 B(M) US', 'Color:': ' Wolf Grey/...",J. Avila,My favorite cross trainers!,Comfortable,1530144000,NaN,NaN
3173,5,True,"06 25, 2018",A1GTC5EVSJNCQ8,B005AGO4LU,"{'Size:': ' 8 B(M) US', 'Color:': ' Blue Tint/...",Amazon Customer,Love them fit perfect,Five Stars,1529884800,NaN,NaN
3174,5,True,"06 20, 2018",A311XHHLM12MUT,B005AGO4LU,"{'Size:': ' 9 B(M) US', 'Color:': ' Blue Tint/...",Peter,Favorite Nike shoe ever! The flex sole is exce...,Love them!,1529452800,NaN,NaN


In [3]:
# Drop unnecessary columns
df.drop(['verified', 'reviewerID', 'asin', 'reviewerName', 'summary', 'unixReviewTime', 'vote', 'image'], axis=1, inplace=True)

df

,overall,reviewTime,style,reviewText
0,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}",Great product and price!
1,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Black (3746...",Great product and price!
2,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...",Great product and price!
3,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue (37867...",Great product and price!
4,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}",Great product and price!
...,...,...,...,...
3171,5,"07 2, 2018","{'Size:': ' 8.5 B(M) US', 'Color:': ' Green Gl...",Perfect fit!
3172,5,"06 28, 2018","{'Size:': ' 5 B(M) US', 'Color:': ' Wolf Grey/...",My favorite cross trainers!
3173,5,"06 25, 2018","{'Size:': ' 8 B(M) US', 'Color:': ' Blue Tint/...",Love them fit perfect
3174,5,"06 20, 2018","{'Size:': ' 9 B(M) US', 'Color:': ' Blue Tint/...",Favorite Nike shoe ever! The flex sole is exce...


In [4]:
# Handle missing values
df.dropna(inplace=True)

df

,overall,reviewTime,style,reviewText
0,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}",Great product and price!
1,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Black (3746...",Great product and price!
2,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...",Great product and price!
3,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue (37867...",Great product and price!
4,5,"09 4, 2015","{'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}",Great product and price!
...,...,...,...,...
3171,5,"07 2, 2018","{'Size:': ' 8.5 B(M) US', 'Color:': ' Green Gl...",Perfect fit!
3172,5,"06 28, 2018","{'Size:': ' 5 B(M) US', 'Color:': ' Wolf Grey/...",My favorite cross trainers!
3173,5,"06 25, 2018","{'Size:': ' 8 B(M) US', 'Color:': ' Blue Tint/...",Love them fit perfect
3174,5,"06 20, 2018","{'Size:': ' 9 B(M) US', 'Color:': ' Blue Tint/...",Favorite Nike shoe ever! The flex sole is exce...


In [5]:
# Handle missing values in the 'style' column
if 'style' in df.columns:
    df.dropna(subset=['style'], inplace=True)

    # Extract relevant features from 'style' column
    def extract_features(style):
        if pd.isna(style):
            return '', ''
        try:
            if isinstance(style, str):
                # Convert string representation of dictionary to actual dictionary
                style_dict = eval(style.replace("'", '"'))
            else:
                # Handle case where the style is already a dictionary
                style_dict = style
            size = style_dict.get('Size:', '').strip()
            color = style_dict.get('Color:', '').strip()
            return size, color
        except Exception as e:
            print(f"Error processing style: {style} with error {e}")  # Debug print
            return '', ''

    # Apply the extraction function to the 'style' column
    df['Size'], df['Color'] = zip(*df['style'].map(extract_features))
    df.drop('style', axis=1, inplace=True)
else:
    print("'style' column is missing in the DataFrame")

In [6]:
df

,overall,reviewTime,reviewText,Size,Color
0,5,"09 4, 2015",Great product and price!,Big Boys,Blue/Orange
1,5,"09 4, 2015",Great product and price!,Big Boys,Black (37467610) / Red/White
2,5,"09 4, 2015",Great product and price!,Big Boys,Blue/Gray Logo
3,5,"09 4, 2015",Great product and price!,Big Boys,Blue (37867638-99) / Yellow
4,5,"09 4, 2015",Great product and price!,Big Boys,Blue/Pink
...,...,...,...,...,...
3171,5,"07 2, 2018",Perfect fit!,8.5 B(M) US,Green Glow/Seaweed - Hasta - White
3172,5,"06 28, 2018",My favorite cross trainers!,5 B(M) US,Wolf Grey/Black-pink Blast/White
3173,5,"06 25, 2018",Love them fit perfect,8 B(M) US,Blue Tint/Green Glow/Hasta/White
3174,5,"06 20, 2018",Favorite Nike shoe ever! The flex sole is exce...,9 B(M) US,Blue Tint/Green Glow/Hasta/White


In [7]:
# Clean and preprocess 'reviewText' column
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\W', ' ', text)     # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)    # Remove extra whitespace
    text = text.lower().strip()        # Convert to lowercase and strip
    text_tokens = word_tokenize(text)   # Tokenize text
    filtered_words = [word for word in text_tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(filtered_words)

df['reviewText'] = df['reviewText'].apply(clean_text)

df

,overall,reviewTime,reviewText,Size,Color
0,5,"09 4, 2015",great product price,Big Boys,Blue/Orange
1,5,"09 4, 2015",great product price,Big Boys,Black (37467610) / Red/White
2,5,"09 4, 2015",great product price,Big Boys,Blue/Gray Logo
3,5,"09 4, 2015",great product price,Big Boys,Blue (37867638-99) / Yellow
4,5,"09 4, 2015",great product price,Big Boys,Blue/Pink
...,...,...,...,...,...
3171,5,"07 2, 2018",perfect fit,8.5 B(M) US,Green Glow/Seaweed - Hasta - White
3172,5,"06 28, 2018",favorite cross trainers,5 B(M) US,Wolf Grey/Black-pink Blast/White
3173,5,"06 25, 2018",love fit perfect,8 B(M) US,Blue Tint/Green Glow/Hasta/White
3174,5,"06 20, 2018",favorite nike shoe ever flex sole excellent so...,9 B(M) US,Blue Tint/Green Glow/Hasta/White


In [8]:
# Convert 'reviewTime' to datetime
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y')

# Convert 'overall' ratings to integer
df['overall'] = df['overall'].astype(int)

df

,overall,reviewTime,reviewText,Size,Color
0,5,2015-09-04,great product price,Big Boys,Blue/Orange
1,5,2015-09-04,great product price,Big Boys,Black (37467610) / Red/White
2,5,2015-09-04,great product price,Big Boys,Blue/Gray Logo
3,5,2015-09-04,great product price,Big Boys,Blue (37867638-99) / Yellow
4,5,2015-09-04,great product price,Big Boys,Blue/Pink
...,...,...,...,...,...
3171,5,2018-07-02,perfect fit,8.5 B(M) US,Green Glow/Seaweed - Hasta - White
3172,5,2018-06-28,favorite cross trainers,5 B(M) US,Wolf Grey/Black-pink Blast/White
3173,5,2018-06-25,love fit perfect,8 B(M) US,Blue Tint/Green Glow/Hasta/White
3174,5,2018-06-20,favorite nike shoe ever flex sole excellent so...,9 B(M) US,Blue Tint/Green Glow/Hasta/White


In [9]:
# Assuming 'reviewText' is the input text and 'overall' is the target label
X = df['reviewText'].values
y = df['overall'].values

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to tensor
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)

In [10]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize and encode sequences
def tokenize_and_encode(texts, max_len=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Tokenize and encode training set
train_input_ids, train_attention_masks = tokenize_and_encode(X_train)
val_input_ids, val_attention_masks = tokenize_and_encode(X_val)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yasmine\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
# Create TensorDatasets
train_dataset = TensorDataset(train_input_ids, train_attention_masks, y_train)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, y_val)

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

In [12]:
# Load pre-trained BERT model with a sequence classification head
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=5,  # Assuming 5 classes for 'overall' ratings
    output_attentions=False,
    output_hidden_states=False,
)

# Send model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
unique_labels = set()
for batch in train_dataloader:
    labels = batch[2]
    unique_labels.update(labels.tolist())

print(f"Unique labels in the dataset: {unique_labels}")
print(f"Number of unique labels: {len(unique_labels)}")

Unique labels in the dataset: {1, 2, 3, 4, 5}
Number of unique labels: 5


In [18]:
def adjust_labels(batch):
    input_ids, attention_mask, labels = batch
    labels = labels - 1  # Adjust labels to be zero-indexed
    return input_ids, attention_mask, labels

train_dataloader = [adjust_labels(batch) for batch in train_dataloader]
val_dataloader = [adjust_labels(batch) for batch in val_dataloader]

In [19]:
from transformers import BertForSequenceClassification

num_labels = 5  # Number of classes after adjusting to zero-indexed labels

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [20]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}'):
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2].long()  # Convert labels to LongTensor
        }
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss}')

# Evaluation on validation set
model.eval()
val_loss, val_accuracy = 0, 0
predictions, true_labels = [], []

for batch in tqdm(val_dataloader, desc='Evaluating'):
    batch = tuple(t.to(device) for t in batch)
    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2].long()  # Convert labels to LongTensor
    }
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    loss = outputs.loss
    val_loss += loss.item()
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.extend(logits.argmax(axis=-1))
    true_labels.extend(label_ids)

# Compute metrics
val_loss /= len(val_dataloader)
val_accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')
print(f'F1 Score: {f1}')

# Classification report and confusion matrix
print(classification_report(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))

Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 155/155 [37:11<00:00, 14.40s/it]


Average training loss: 1.824998959418266


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 155/155 [34:42<00:00, 13.43s/it]


Average training loss: 1.8256525193491289


Epoch 3: 100%|███████████████████████████████████████████████████████████████████████| 155/155 [32:38<00:00, 12.64s/it]


Average training loss: 1.8217311682239656


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [03:03<00:00,  4.70s/it]
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 1.7923405476105518
Validation Accuracy: 0.0791599353796446
F1 Score: 0.02401273642576722
              precision    recall  f1-score   support

           0       0.02      0.23      0.03        22
           1       0.00      0.00      0.00        16
           2       0.13      0.66      0.21        67
           3       0.00      0.00      0.00        92
           4       0.00      0.00      0.00       422

    accuracy                           0.08       619
   macro avg       0.03      0.18      0.05       619
weighted avg       0.01      0.08      0.02       619

[[  5   0  17   0   0]
 [  3   0  13   0   0]
 [ 23   0  44   0   0]
 [ 44   0  48   0   0]
 [193   0 229   0   0]]


C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Define the number of epochs and batch size
epochs = 5
batch_size = 16

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}'):
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2].long()  # Convert labels to LongTensor
        }
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average training loss: {avg_train_loss}')

    # Evaluation on validation set
    model.eval()
    val_loss, val_accuracy = 0, 0
    predictions, true_labels = [], []

    for batch in tqdm(val_dataloader, desc='Evaluating'):
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2].long()  # Convert labels to LongTensor
        }
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        loss = outputs.loss
        val_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.extend(logits.argmax(axis=-1))
        true_labels.extend(label_ids)

    # Compute metrics
    val_loss /= len(val_dataloader)
    val_accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')

    print(f'Validation Loss: {val_loss}')
    print(f'Validation Accuracy: {val_accuracy}')
    print(f'F1 Score: {f1}')

    # Classification report and confusion matrix
    print(classification_report(true_labels, predictions))
    print(confusion_matrix(true_labels, predictions))

    model.train()

# Final evaluation on the test set (if available)
# Assuming you have a test_dataloader for this purpose
test_loss, test_accuracy = 0, 0
test_predictions, test_true_labels = [], []

for batch in tqdm(test_dataloader, desc='Testing'):
    batch = tuple(t.to(device) for t in batch)
    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2].long()  # Convert labels to LongTensor
    }
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    loss = outputs.loss
    test_loss += loss.item()
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    test_predictions.extend(logits.argmax(axis=-1))
    test_true_labels.extend(label_ids)

# Compute metrics on the test set
test_loss /= len(test_dataloader)
test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_f1 = f1_score(test_true_labels, test_predictions, average='weighted')

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Test F1 Score: {test_f1}')

# Classification report and confusion matrix for the test set
print(classification_report(test_true_labels, test_predictions))
print(confusion_matrix(test_true_labels, test_predictions))

Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 155/155 [30:31<00:00, 11.82s/it]


Epoch 1/5, Average training loss: 1.826211703208185


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [03:05<00:00,  4.76s/it]
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being s

Validation Loss: 1.7923405476105518
Validation Accuracy: 0.0791599353796446
F1 Score: 0.02401273642576722
              precision    recall  f1-score   support

           0       0.02      0.23      0.03        22
           1       0.00      0.00      0.00        16
           2       0.13      0.66      0.21        67
           3       0.00      0.00      0.00        92
           4       0.00      0.00      0.00       422

    accuracy                           0.08       619
   macro avg       0.03      0.18      0.05       619
weighted avg       0.01      0.08      0.02       619

[[  5   0  17   0   0]
 [  3   0  13   0   0]
 [ 23   0  44   0   0]
 [ 44   0  48   0   0]
 [193   0 229   0   0]]


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 155/155 [33:05<00:00, 12.81s/it]


...

Epoch 3/5, Average training loss: 1.8257106842533235


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [04:09<00:00,  6.39s/it]
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yasmine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being s

Validation Loss: 1.7923405476105518
Validation Accuracy: 0.0791599353796446
F1 Score: 0.02401273642576722
              precision    recall  f1-score   support

           0       0.02      0.23      0.03        22
           1       0.00      0.00      0.00        16
           2       0.13      0.66      0.21        67
           3       0.00      0.00      0.00        92
           4       0.00      0.00      0.00       422

    accuracy                           0.08       619
   macro avg       0.03      0.18      0.05       619
weighted avg       0.01      0.08      0.02       619

[[  5   0  17   0   0]
 [  3   0  13   0   0]
 [ 23   0  44   0   0]
 [ 44   0  48   0   0]
 [193   0 229   0   0]]


Epoch 4:  29%|███████████████████▏                                              | 45/155 [6:35:46<16:07:26, 527.70s/it]


KeyboardInterrupt: 